In [1]:
import os
from pathlib import Path

if "PROJECT_ROOT" not in globals():
    PROJECT_ROOT = Path.cwd().parent.resolve()

os.chdir(PROJECT_ROOT)

In [2]:
import pandas as pd
from utils import ModelForAnalysis

In [3]:
RESULTS_DIR = PROJECT_ROOT/"analysis_results"

LARGELY_SAMPLED_EPITOPES = pd.read_csv("analysis_results/CDR3 Levenshtein/ovr_nn_200_shot.csv").epitope.unique()

NUM_SHOTS_OF_INTEREST = [1,2,5,10,20,50,100,200]

In [4]:
models = (
    ModelForAnalysis("SCEPTR", "ovr_nn", "#7048e8", "d"),
    ModelForAnalysis("SCEPTR (synthetic data)", "ovr_nn", "#7048e8", "d"),
    ModelForAnalysis("SCEPTR (shuffled data)", "ovr_nn", "#7048e8", "d"),
)

raw_results = [model.load_data(200) for model in models]
summary_per_model = [df.groupby("epitope").aggregate({"auc": "mean"}) for df in raw_results]
summary_per_model = [s[s.index.isin(LARGELY_SAMPLED_EPITOPES)] for s in summary_per_model]

for model, df in zip(models, summary_per_model):
    df.columns = [model.name]

summary = pd.concat(summary_per_model, axis="columns")

In [ ]:
summary